## crawler3

In [1]:
import requests
import lxml.html

In [2]:
def main():
    session = requests.Session()
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    
    for url in urls:
        print(url)

In [3]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url
        
if __name__ == '__main__':
    main()

https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938
https://www.hanbit.co.kr/store/books/look.php?p_code=B6084607806
https://www.hanbit.co.kr/store/books/look.php?p_code=B8948111854
https://www.hanbit.co.kr/store/books/look.php?p_code=B8661740335
https://www.hanbit.co.kr/store/books/look.php?p_code=B1382037871
https://www.hanbit.co.kr/store/books/look.php?p_code=B2847674054
https://www.hanbit.co.kr/store/books/look.php?p_code=B4162156367
https://www.hanbit.co.kr/store/books/look.php?p_code=B5156679966
https://www.hanbit.co.kr/store/books/look.php?p_code=B6189263297
https://www.hanbit.co.kr/store/books/look.php?p_code=B4813114979
https://www.hanbit.co.kr/store/books/look.php?p_code=B8806843023
https://www.hanbit.co.kr/store/books/look.php?p_code=B2859439220
https://www.hanbit.co.kr/store/books/look.php?p_code=B1996635146
https://www.hanbit.co.kr/store/books/look.php?p_code=B9063949643
https://www.hanbit.co.kr/store/books/look.php?p_code=B7400451696
https://www.hanbit.co.kr/

## crawler4

In [12]:
import requests
import lxml.html

def main():
    session = requests.Session()
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break
        
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [13]:
def scrape_detail_page(response):
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url':response.url,
        'title':root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price':root.cssselect('.pbr strong')[0].text_content(),
        'content':[p.text_content()\
                  for p in root.cssselect('#tabs_3 .hanbit_edit_view p')]
    }
    return ebook

if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['', '\r\n\t\t<1편>\r\n', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '\xa0', '\r\n\t\t<2편>\r\n', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '\xa0', '\r\n\t\t<3편>\r\n', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '\xa0', '\r\n\t\t<4편>\r\n', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}


## crawler5

In [11]:
import re
import requests
import lxml.html

def main():
    session = requests.Session()
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    
    for url in urls:
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break

        
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

           
        
def scrape_detail_page(response):
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url':response.url,
        'title':root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price':root.cssselect('.pbr strong')[0].text_content(),
        'content':[normalize_spaces(p.text_content())
                  for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
                  if normalize_spaces(p.text_content()) != '']
    }
    return ebook

def normalize_spaces(s):
    return re.sub(r'\s+', ' ', s).strip()

if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['<1편>', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '<2편>', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '<3편>', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '<4편>', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}


## crawler final

In [17]:
import time
import re
import requests
import lxml.html

In [23]:
def main():
    session = requests.Session()
    response = session.get('http://www.hanbit.co.kr/store/books/new_book_list.html')
    urls = scrape_list_page(response)
    for url in urls:
        time.sleep(1)
        response = session.get(url)
        ebook = scrape_detail_page(response)
        print(ebook)
        break

In [19]:
def scrape_list_page(response):
    root = lxml.html.fromstring(response.content)
    root.make_links_absolute(response.url)
    for a in root.cssselect('.view_box .book_tit a'):
        url = a.get('href')
        yield url

In [20]:
def scrape_detail_page(response):
    root = lxml.html.fromstring(response.content)
    ebook = {
        'url':response.url,
        'title':root.cssselect('.store_product_info_box h3')[0].text_content(),
        'price':root.cssselect('.pbr strong')[0].text_content(),
        'content':[normalize_spaces(p.text_content())
                  for p in root.cssselect('#tabs_3 .hanbit_edit_view p')
                  if normalize_spaces(p.text_content()) != '']
    }
    return ebook

In [21]:
def normalize_spaces(s):
    return re.sub(r'\s+', ' ', s).strip()

In [25]:
if __name__ == '__main__':
    main()

{'url': 'https://www.hanbit.co.kr/store/books/look.php?p_code=B3178834938', 'title': '받침 없는 한글 동화 : 4권 세트', 'price': '49,680', 'content': ['<1편>', '이렇게 읽어요', '1. 무시무시 마녀가 이사 와!', '2. 아기 고래 뿌우의 노래', '<2편>', '이렇게 읽어요', '1. 도, 도, 도깨비다!', '2. 느티나무 아파트', '<3편>', '이렇게 읽어요', '1. 바쁘다 바빠 너구리 바빠', '2. 꼬마 지우개 마구마구', '<4편>', '이렇게 읽어요', '1. 도깨비 파자마 파티', '2. 꼬마 두루미 뚜루']}
